In [1]:
import pystac_client
import stackstac
import planetary_computer as pc

# Important infor for later
endpoint = "https://planetarycomputer.microsoft.com/api/stac/v1"
collection = "sentinel-2-l2a"

west_lon_degree=6.7319
south_lat_degree=51.1742
east_lon_degree=6.9013
north_lat_degree=51.0652


bounds = (west_lon_degree, south_lat_degree, east_lon_degree, north_lat_degree)

# Open the STAC
catalog = pystac_client.Client.open(endpoint, modifier=pc.sign_inplace)

# Define your area
area_of_interest = {
    "type": "Polygon",
    "coordinates": [
        [
            [bounds[0], bounds[1]],
            [bounds[2], bounds[1]],
            [bounds[2], bounds[3]],
            [bounds[0], bounds[3]],
            [bounds[0], bounds[1]],
        ]
    ],
}

# Search the items
items = catalog.search(
    collections=[collection],
    intersects=area_of_interest,
    datetime="2019-06-01/2019-08-01",
    query={"eo:cloud_cover": {"lt"  : 10}},
).get_all_items()


In [13]:
from pyproj import CRS
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info

utm_crs_list = query_utm_crs_info(
    datum_name="WGS 84",
    area_of_interest=AreaOfInterest(
        west_lon_degree=west_lon_degree,
        south_lat_degree=south_lat_degree,
        east_lon_degree=east_lon_degree,
        north_lat_degree=north_lat_degree,
    ),
)
utm_crs = CRS.from_epsg(utm_crs_list[0].code)
utm_crs.to_epsg()

32632

In [14]:
URL = "https://earth-search.aws.element84.com/v0"
catalog = pystac_client.Client.open(URL)

stac_items = catalog.search(
    intersects=dict(type="Point", coordinates=[6.7319, 51.1742]),
    collections=["sentinel-s2-l2a-cogs"],
    datetime="2020-04-01/2020-05-01"
).get_all_items()

print(stac_items.items[0].to_dict)
stack = stackstac.stack(stac_items,epsg=utm_crs.to_epsg())
print(stack)


<bound method Item.to_dict of <Item id=S2A_31UGS_20200427_0_L2A>>
<xarray.DataArray 'stackstac-21dda693c65eae318dca24b000e9cc86' (time: 12,
                                                                band: 17,
                                                                y: 11831,
                                                                x: 13597)>
dask.array<fetch_raster_window, shape=(12, 17, 11831, 13597), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/24)
  * time                        (time) datetime64[ns] 2020-04-02T10:46:47 ......
    id                          (time) <U24 'S2B_32ULB_20200402_0_L2A' ... 'S...
  * band                        (band) <U8 'overview' 'B11' ... 'visual' 'SCL'
  * x                           (x) float64 2.738e+05 2.738e+05 ... 4.098e+05
  * y                           (y) float64 5.701e+06 5.701e+06 ... 5.582e+06
    sentinel:product_id         (time) <U60 'S2B_MSIL2A_20200402T103619_N0214...
    ..